In [1]:
from imblearn.under_sampling import AllKNN as AKNN
from imblearn.under_sampling import ClusterCentroids as CC
from imblearn.under_sampling import CondensedNearestNeighbour as CNN
from imblearn.under_sampling import EditedNearestNeighbours as ENN
from imblearn.under_sampling import InstanceHardnessThreshold as IHT
from imblearn.under_sampling import RepeatedEditedNearestNeighbours as RENN
from imblearn.under_sampling import NearMiss as NM
from imblearn.under_sampling import NeighbourhoodCleaningRule as NCL
from imblearn.under_sampling import OneSidedSelection as OSS
from imblearn.under_sampling import RandomUnderSampler as RUS
from imblearn.under_sampling import TomekLinks as TL
from pandas import read_csv
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.svm import SVC as SVM

In [3]:
import imblearn.metrics
import sklearn.metrics

from collections import Counter


def metric_decorator(metric_function):
    def metric_wrapper(ground_truth, predictions, minority_class=None):
        if minority_class is None:
            minority_class = Counter(ground_truth).most_common()[-1][0]

        return metric_function(ground_truth, predictions, minority_class)

    return metric_wrapper


@metric_decorator
def precision(ground_truth, predictions, minority_class=None):
    return sklearn.metrics.precision_score(ground_truth, predictions, pos_label=minority_class)


@metric_decorator
def recall(ground_truth, predictions, minority_class=None):
    return sklearn.metrics.recall_score(ground_truth, predictions, pos_label=minority_class)


@metric_decorator
def f_measure(ground_truth, predictions, minority_class=None):
    return sklearn.metrics.f1_score(ground_truth, predictions, pos_label=minority_class)


def g_mean(ground_truth, predictions):
    return imblearn.metrics.geometric_mean_score(ground_truth, predictions)


def auc(ground_truth, predictions):
    return sklearn.metrics.roc_auc_score(ground_truth, predictions)

In [4]:
df=pd.read_csv('bs140513_032310.csv')
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df.loc[:,['customer']]=le.fit_transform(df.loc[:,['customer']])
df.loc[:,['age']]=le.fit_transform(df.loc[:,['age']])
df.loc[:,['zipcodeOri']]=le.fit_transform(df.loc[:,['zipcodeOri']])
df.loc[:,['zipMerchant']]=le.fit_transform(df.loc[:,['zipMerchant']])
df.loc[:,['merchant']]=le.fit_transform(df.loc[:,['merchant']])
df.loc[:,['category']]=le.fit_transform(df.loc[:,['category']])
df.loc[:,['gender']]=le.fit_transform(df.loc[:,['gender']])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [5]:
train,test=train_test_split(df,random_state=42)

In [6]:
x_train=train.drop('fraud',axis=1)
y_train=train.fraud
xtest=test.drop('fraud',axis=1)
ytest=test.fraud
n_minority = Counter(y_train).most_common()[1][1]
n_majority = Counter(y_train).most_common()[0][1]

In [7]:
imblearn_ratios = [((n_majority - n_minority) * ratio + n_minority) / n_majority for ratio in [0.5, 0.75, 1.0]]

In [8]:
imblearn_ratios

[0.5061857027412311, 0.7530928513706155, 1.0]

In [65]:
samtec={
#          'AKNN':AKNN(n_neighbors=3)
#         'CC':CC(sampling_strategy=0.5)
#         'CNN':CNN(n_neighbors=3)
#         'ENN':ENN(n_neighbors=3)
#         'IHT':IHT(sampling_strategy=0.5),    
#         'RENN':RENN(n_neighbors=3)
#         'NM':NM(n_neighbors=3)
#         'NCL':NCL(n_neighbors=3)
#           'OSS':OSS(n_neighbors=3)
        'RUS':RUS(sampling_strategy=0.5)
#         'TL':TL()
        }

In [31]:
for i in clf:
    x=clf[i]
    print(x)


XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)
KNeighborsClassifier()
LinearRegression()
LinearDiscriminantAnalysis()
SVC()
RandomForestClassifier(max_features='sqrt')


In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [14]:
from keras.models import Sequential
from keras.layers import Dense
def model_dnn(look_back):
    model=Sequential()
    model.add(Dense(units=32, input_dim=look_back, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',  optimizer='adam')
    return model


In [13]:
clf = {'XGB':XGBClassifier(),
       'KNN': KNN(),
       'SVM': SVM(gamma='scale'),
       'RF':RandomForestClassifier(n_estimators=100, bootstrap = True,max_features = 'sqrt')}

In [82]:
x=pd.DataFrame(columns=[cld,'precision','recall','f_measure','auc'])

In [117]:
sampl=samtec['RUS']
Xtrain,Ytrain=sampl.fit_resample(x_train,y_train)


In [118]:
classifier=clf['XGB']
classifier.fit(Xtrain,Ytrain)
yhat=classifier.predict(xtest)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:59:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [107]:
import numpy as np

In [109]:
yhat=np.round(yhat)

In [111]:
yhat.astype(int)

array([0, 0, 0, ..., 0, 0, 0])

In [124]:
precision((ytest),yhat)

0.32916024653312786

In [66]:
samtec

{'RUS': RandomUnderSampler(sampling_strategy=0.5)}

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1))

In [18]:
x_train=scaling.fit_transform(x_train)
xtest=scaling.fit_transform(xtest)

In [67]:
result=dict()
for i in samtec:
    sampl=samtec[i]
    metrices=pd.DataFrame(columns=['Classifier','precision','recall','f_measure','auc','TP','FP','TN','FN'])
    Xtrain,Ytrain=sampl.fit_resample(x_train,y_train)
    x=0
    for j in clf:
        classifier=clf[j]
        classifier.fit(Xtrain,Ytrain)
        yhat=classifier.predict(xtest)
        tn, fp, fn, tp = confusion_matrix(ytest,yhat).ravel()
        metrices.loc[x, ['Classifier']]=j
        metrices.loc[x, ['precision']]=precision(ytest,yhat)
        metrices.loc[x, ['recall']]=recall(ytest,yhat)
        metrices.loc[x, ['f_measure']]=f_measure(ytest,yhat)
        metrices.loc[x, ['auc']]=auc(ytest,yhat)
        metrices.loc[x, ['TP']]=tp
        metrices.loc[x, ['TN']]=tn
        metrices.loc[x, ['FN']]=fn
        metrices.loc[x, ['FP']]=fp
        x=x+1
        print(j)
    
    result[i]=metrices
    print(i)
    print(result[i])
    
    
        
        
        
        
        
    
    
    

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:08:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGB
KNN
SVM
RF
RUS
  Classifier precision    recall f_measure       auc    TP    FP      TN   FN
0        XGB  0.326597  0.978857  0.489778  0.977408  1713  3532  143379   37
1        KNN  0.194083  0.937143  0.321569  0.945394  1640  6810  140101  110
2        SVM  0.206502  0.929143  0.337905  0.943307  1626  6248  140663  124
3         RF  0.307291  0.975429  0.467351  0.974618  1707  3848  143063   43


In [68]:
metrices.to_csv('RUS.csv')

In [59]:
metrices

,Classifier,precision,recall,f_measure,auc,TP,FP,TN,FN
0,XGB,0.844419,0.834286,0.839322,0.916227,1460,269,146642,290
1,KNN,0.713961,0.537714,0.613429,0.767574,941,377,146534,809
2,SVM,0.932458,0.568,0.705966,0.783755,994,72,146839,756
3,RF,0.819663,0.805143,0.812338,0.901516,1409,310,146601,341


In [30]:
result['NCL'].to_csv('NCL.csv')

In [98]:
metrices

,Classifier,precision,recall,f_measure,auc
0,XGB,0.749149,0.880571,0.809561,0.93853
1,KNN,0.558105,0.639429,0.596005,0.816699
2,LR,NaN,NaN,NaN,NaN


In [41]:
sampl=samtec['RUS']

In [42]:
Xtrain,Ytrain=sampl.fit_resample(scaling.fit_transform(x_train),y_train)

In [65]:
classifier=clf['DNN']

In [16]:
xtest=scaling.fit_transform(xtest)

In [66]:
classifier.fit(Xtrain,Ytrain,epochs=10, batch_size=30, verbose=1)

Epoch 1/10
16350/16350 [==============================] - 1s 87us/step - loss: 0.0293
Epoch 2/10
16350/16350 [==============================] - 1s 76us/step - loss: 0.0289
Epoch 3/10
16350/16350 [==============================] - ETA: 0s - loss: 0.028 - 1s 80us/step - loss: 0.0284
Epoch 4/10
16350/16350 [==============================] - 1s 78us/step - loss: 0.0281
Epoch 5/10
16350/16350 [==============================] - 1s 78us/step - loss: 0.0283
Epoch 6/10
16350/16350 [==============================] - 1s 82us/step - loss: 0.0275
Epoch 7/10
16350/16350 [==============================] - 1s 83us/step - loss: 0.0275
Epoch 8/10
16350/16350 [==============================] - 1s 76us/step - loss: 0.0269
Epoch 9/10
16350/16350 [==============================] - 1s 76us/step - loss: 0.0267
Epoch 10/10
16350/16350 [==============================] - 1s 82us/step - loss: 0.0262


In [67]:
yhat=classifier.predict(xtest)

In [70]:
print(precision(ytest,yhat))
print(recall(ytest,yhat))

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [53]:
def print_evaluation_metrics(y_true, y_pred):
        # accuracy: (tp + tn) / (p + n)
        accuracy = accuracy_score(y_true, y_pred)
        print('Accuracy: %f' % accuracy)
        # precision tp / (tp + fp)
        precision = precision_score(y_true, y_pred)
        print('Precision: %f' % precision)
        # recall: tp / (tp + fn)
        recall = recall_score(y_true, y_pred)
        print('Recall: %f' % recall)
        # f1: 2 tp / (2 tp + fp + fn)
        f1 = f1_score(y_true, y_pred)
        print('F1 score: %f' % f1)

In [120]:
print_evaluation_metrics(ytest,yhat)

Accuracy: 0.976295
Precision: 0.329160
Recall: 0.976571


NameError: name 'f1_score' is not defined